In [1]:
import pennylane as qml
import numpy as np
import sys

%load_ext autoreload
%autoreload 2

In [2]:
from pennylane_ls import *
from pennylane_ls.MultiQuditDevice import MultiQuditDevice

In [3]:
from heroku_credentials import username, password
#from local_credentials  import username, password

In [13]:
multiDevice = MultiQuditDevice(wires=5,shots=5, username = username, password = password)

In [14]:
multiDevice.capabilities()

{'model': 'qudit',
 'supports_finite_shots': True,
 'supports_tensor_observables': True,
 'returns_probs': False}

We build up the circuit for the QAOA in Pennylane. We do this (for now) for a fixed number of qudits and fixed dimension d. We take the following instance of our benchmark suite:

exp_dict['ExpA1']={ 'jm':[0,1],\
                    'pv':[0,1],\
                    'emax':1,\
                    'emin':0 }

In [15]:
@qml.qnode(multiDevice)
def quantum_circuit_for_load(alpha=0):
    load(100,wires=0)
    load(20,wires=1)
    return qml.expval(MultiQuditOps.Z(0))

In [16]:
res = quantum_circuit_for_load()
res

NotImplementedError: 

In [ ]:
exp_dict = dict()

# --> z=0, j=[0,1]
exp_dict['ExpA1']={ 'jm':[0,1],\
                    'pv':[0,1],\
                    'emax':1,\
                    'emin':0 }

In [ ]:
quantum_circuit_simpleQAOA(1,2)